<a href="https://colab.research.google.com/github/tausif04/Data-Mining-and-Machine-Learning-Lab/blob/main/SVM_KNN_Naive_Bias_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier ,KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
df=pd.read_csv('/content/drive/MyDrive/My Dataset/student_habits_performance.csv')
df

,student_id,age,gender,study_hours_per_day,social_media_hours,netflix_hours,part_time_job,attendance_percentage,sleep_hours,diet_quality,exercise_frequency,parental_education_level,internet_quality,mental_health_rating,extracurricular_participation,exam_score
0,S1000,23,Female,0.0,1.2,1.1,No,85.0,8.0,Fair,6,Master,Average,8,Yes,56.2
1,S1001,20,Female,6.9,2.8,2.3,No,97.3,4.6,Good,6,High School,Average,8,No,100.0
2,S1002,21,Male,1.4,3.1,1.3,No,94.8,8.0,Poor,1,High School,Poor,1,No,34.3
3,S1003,23,Female,1.0,3.9,1.0,No,71.0,9.2,Poor,4,Master,Good,1,Yes,26.8
4,S1004,19,Female,5.0,4.4,0.5,No,90.9,4.9,Fair,3,Master,Good,1,No,66.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,S1995,21,Female,2.6,0.5,1.6,No,77.0,7.5,Fair,2,High School,Good,6,Yes,76.1
996,S1996,17,Female,2.9,1.0,2.4,Yes,86.0,6.8,Poor,1,High School,Average,6,Yes,65.9
997,S1997,20,Male,3.0,2.6,1.3,No,61.9,6.5,Good,5,Bachelor,Good,9,Yes,64.4
998,S1998,24,Male,5.4,4.1,1.1,Yes,100.0,7.6,Fair,0,Bachelor,Average,1,No,69.7


In [54]:
value_40= (40 - df['exam_score'].mean()) / df['exam_score'].std()
value_50= (50 - df['exam_score'].mean()) / df['exam_score'].std()
value_60= (60 - df['exam_score'].mean()) / df['exam_score'].std()
value_70= (70 - df['exam_score'].mean()) / df['exam_score'].std()
value_80= (80 - df['exam_score'].mean()) / df['exam_score'].std()

In [55]:
print(df.isnull().sum())

student_id                        0
age                               0
gender                            0
study_hours_per_day               0
social_media_hours                0
netflix_hours                     0
part_time_job                     0
attendance_percentage             0
sleep_hours                       0
diet_quality                      0
exercise_frequency                0
parental_education_level         91
internet_quality                  0
mental_health_rating              0
extracurricular_participation     0
exam_score                        0
dtype: int64


In [56]:
df = df.drop(columns=['student_id'])


In [57]:
#manually mapping
diet_quality_order = ['Poor', 'Fair', 'Good']
internet_quality_order = ['Poor', 'Average', 'Good']
parent_edu_order = ['High School', 'Associate', 'Bachelor', 'Master']

#ordinal encodingS
df['diet_quality'] = df['diet_quality'].map({k: i for i, k in enumerate(diet_quality_order)})
df['internet_quality'] = df['internet_quality'].map({k: i for i, k in enumerate(internet_quality_order)})
df['parental_education_level'] = df['parental_education_level'].map({k: i for i, k in enumerate(parent_edu_order)})

In [58]:
#Binary Encoding
df.columns = df.columns.str.strip()
df['part_time_job'] = df['part_time_job'].map({'Yes': 1, 'No': 0})
df['extracurricular_participation'] = df['extracurricular_participation'].map({'Yes': 1, 'No': 0})
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0, 'Other': 2})


In [59]:
# Feature engineering
df['total_screen_time'] = df['social_media_hours'] + df['netflix_hours']
df['sleep_sufficient'] = df['sleep_hours'].apply(lambda x: 1 if x >= 7 else 0)

In [60]:
from sklearn.preprocessing import StandardScaler
numeric_features = [
    'age', 'study_hours_per_day', 'social_media_hours', 'netflix_hours',
    'attendance_percentage', 'sleep_hours', 'exercise_frequency',
    'mental_health_rating', 'exam_score', 'total_screen_time'
]

# Standard scaling
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [61]:
print(df)

          age  gender  study_hours_per_day  social_media_hours  netflix_hours  \
0    1.084551       0            -2.418068           -1.114064      -0.669750   
1   -0.215870       0             2.281707            0.251315       0.446965   
2    0.217604       1            -1.464491            0.507324      -0.483631   
3    1.084551       0            -1.736942            1.190013      -0.762810   
4   -0.649344       0             0.987566            1.616694      -1.228108   
..        ...     ...                  ...                 ...            ...   
995  0.217604       0            -0.647139           -1.711417      -0.204452   
996 -1.516291       0            -0.442801           -1.284736       0.540025   
997 -0.215870       1            -0.374688            0.080643      -0.483631   
998  1.518025       1             1.260017            1.360685      -0.669750   
999 -0.649344       0             0.510777            0.336651       0.074727   

     part_time_job  attenda

In [62]:
# KNN Regression
data_classifier = df.copy()
features =  ['age','study_hours_per_day','attendance_percentage','diet_quality','sleep_sufficient', 'internet_quality','exercise_frequency', 'part_time_job','extracurricular_participation','mental_health_rating','total_screen_time']
target_classifier = ['exam_score']
X = data_classifier[features]
y = data_classifier[target_classifier]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(x_train, y_train)

y_pred_knn = knn_reg.predict(x_test)
print("KNN Regression Results")
print(y_pred_knn[:10])
print("*"*20)
print(y_test[:10])
print("RMSE:", sqrt(mean_squared_error(y_test, y_pred_knn)))


KNN Regression Results
[[-0.43018065]
 [ 0.34469547]
 [ 0.65275019]
 [-0.12449558]
 [-0.23468438]
 [-1.19083887]
 [ 0.26649696]
 [-0.75245329]
 [-1.30339732]
 [ 0.15156885]]
********************
     exam_score
521   -0.319992
737    0.183559
740    0.556779
660    0.586400
411   -0.675440
678   -0.959798
626    0.071001
513   -0.420702
859   -1.943203
136   -0.118571
RMSE: 0.4695894839950791


In [63]:
#KNN Classification
def categorize(score):
    if score < value_40:
        return "F"
    elif score >=value_40 and score < value_50:
        return "D"
    elif score >=value_40 and score < value_50:
        return "C"
    elif score >=value_50 and score < value_60:
        return "B"
    elif score >=value_60 and score < value_70:
        return "B+"
    elif score >=value_70 and score < value_80:
        return "A"
    else:
        return "A+"

data_classifier['performance_category'] = data_classifier['exam_score'].apply(categorize)

X_cls = data_classifier[features]
y_cls = data_classifier['performance_category']


X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

knn_cls = KNeighborsClassifier(n_neighbors=5)
knn_cls.fit(X_train_cls, y_train_cls)
y_pred_cls = knn_cls.predict(X_test_cls)

print("KNN Classification Results")
print(y_pred_cls[:10])
print("*"*20)
print(y_test[:10])
print("Accuracy:", accuracy_score(y_test_cls, y_pred_cls))
print("\nClassification Report:\n", classification_report(y_test_cls, y_pred_cls))

KNN Classification Results
['B+' 'A' 'A+' 'B+' 'A' 'B' 'A' 'D' 'D' 'B+']
********************
     exam_score
521   -0.319992
737    0.183559
740    0.556779
660    0.586400
411   -0.675440
678   -0.959798
626    0.071001
513   -0.420702
859   -1.943203
136   -0.118571
Accuracy: 0.495

Classification Report:
               precision    recall  f1-score   support

           A       0.58      0.63      0.60        60
          A+       0.72      0.73      0.73        45
           B       0.23      0.35      0.28        23
          B+       0.35      0.25      0.29        44
           D       0.33      0.32      0.32        19
           F       0.75      0.33      0.46         9

    accuracy                           0.49       200
   macro avg       0.49      0.44      0.45       200
weighted avg       0.50      0.49      0.49       200



In [68]:
# SVM Classification for 'part_time_job'
features =  ['age','study_hours_per_day','attendance_percentage','diet_quality','sleep_sufficient', 'internet_quality','exercise_frequency','extracurricular_participation','mental_health_rating','total_screen_time','exam_score']
target_column = ['part_time_job']
X_svm = data_classifier[features]
y_svm = data_classifier[target_column]


X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_svm, y_svm, test_size=0.2, random_state=42)

svm_model = SVC(kernel='rbf', C=1.0)
svm_model.fit(X_train_svm, y_train_svm)
y_pred_svm = svm_model.predict(X_test_svm)

print("SVM Classification Results for 'part_time_job'")
print(y_pred_svm[:10])
print("*"*20)
print(y_test_svm[:10])
print("Accuracy:", accuracy_score(y_test_svm, y_pred_svm))
print("Classification Report")
print(classification_report(y_test_svm, y_pred_svm))

SVM Classification Results for 'part_time_job'
[0 0 0 0 0 0 0 0 0 0]
********************
     part_time_job
521              0
737              1
740              1
660              0
411              1
678              1
626              0
513              1
859              0
136              0
Accuracy: 0.785
Classification Report
              precision    recall  f1-score   support

           0       0.79      1.00      0.88       157
           1       0.00      0.00      0.00        43

    accuracy                           0.79       200
   macro avg       0.39      0.50      0.44       200
weighted avg       0.62      0.79      0.69       200



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning

In [69]:
# Naive Bayes Classification for 'part_time_job'
target_column = 'part_time_job'
X_nb = data_classifier[features]
y_nb = data_classifier[target_column]


X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_nb, y_nb, test_size=0.2, random_state=42)


nb_model = GaussianNB()
nb_model.fit(X_train_nb, y_train_nb)

y_pred_nb = nb_model.predict(X_test_nb)

print(" Naive Bayes Classification for 'part_time_job'")
print(y_pred_nb[:10])
print("*"*20)
print(y_test_nb[:10])
print("Accuracy:", accuracy_score(y_test_nb, y_pred_nb))
print("\nClassification Report:\n", classification_report(y_test_nb, y_pred_nb))


 Naive Bayes Classification for 'part_time_job'
[0 0 0 0 0 0 0 0 0 0]
********************
521    0
737    1
740    1
660    0
411    1
678    1
626    0
513    1
859    0
136    0
Name: part_time_job, dtype: int64
Accuracy: 0.785

Classification Report:
               precision    recall  f1-score   support

           0       0.79      1.00      0.88       157
           1       0.00      0.00      0.00        43

    accuracy                           0.79       200
   macro avg       0.39      0.50      0.44       200
weighted avg       0.62      0.79      0.69       200



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
